# Flower image classifier

## Project

Machine learning is getting incorporated more and more into everyday apps. Soon, anything with a camera will use image recognition which is based on deep learning models.

The goal of the project is to build such a model for recognizing flower types and use it in a simple application. The app is going to tell the type of a flower that is on an image.